In [1]:
import pandas as pd
from collections import defaultdict
from rapidfuzz import fuzz, process

coaches = pd.read_csv('../data/raw/coaches.csv')
players = pd.read_csv('../data/raw/players.csv')
schools = pd.read_csv('../data/raw/schools.csv')


In [2]:
print(coaches.head())
print(players.head())
print(schools.head())

                  School              Name                    Title
0  University Of Florida  Kevin O'Sullivan               Head Coach
1  University Of Florida   Chuck Jeroloman     Associate Head Coach
2  University Of Florida      Taylor Black          Assistant Coach
3  University Of Florida        David Kopp           Pitching Coach
4  University Of Florida     Michael Byrne  Student Assistant Coach
                  School           Name  Jersey Position Class Year   Height  \
0  University Of Florida  Justin Nadeau       1   INF/OF        Jr.   6' 0''   
1  University Of Florida       Ty Evans       2       OF        Sr.   6' 2''   
2  University Of Florida     Kyle Jones       3       OF        So.   6' 3''   
3  University Of Florida   Cade Kurland       4      INF        Jr.  5' 11''   
4  University Of Florida      Blake Cyr       5   OF/INF        Jr.  5' 11''   

    Weight                   High School  
0  195 lbs  Bartram Trail (Jacksonville)  
1  215 lbs            Lak

In [3]:
counts = players['Name'].value_counts()
multiple_player_names = counts[counts > 1]
multiple_player_names

Name
Joey McMannis      2
Cameron Johnson    2
Name: count, dtype: int64

In [4]:
high_schools = players['High School'].dropna().unique()
len(high_schools)

444

In [5]:
temp_schools = high_schools[:5]
for school in temp_schools:
    if school == 'N/A':
        continue

    matches = process.extract(
        school,
        temp_schools,
        scorer = fuzz.ratio,
        score_cutoff = 80
    )

    print(matches[0])

('Bartram Trail (Jacksonville)', 100.0, 0)
('Lakeland Christian', 100.0, 1)
('North Oconee (Stetson)', 100.0, 2)
('Berkeley Prep', 100.0, 3)
('Windermere (Miami)', 100.0, 4)


In [6]:
unique_schools = defaultdict(list)

for school in high_schools:
    if school == 'N/A':
        continue

    matches = process.extract(
        school,
        unique_schools.keys(),
        scorer = fuzz.token_sort_ratio,
        score_cutoff = 85
    )

    if not matches:
        unique_schools[school].append(school)
    else:
        unique_schools[matches[0][0]].append(school)

for key, vals in unique_schools.items():
    if isinstance(vals, list):
        print(f"{key}: {vals}")

Bartram Trail (Jacksonville): ['Bartram Trail (Jacksonville)']
Lakeland Christian: ['Lakeland Christian']
North Oconee (Stetson): ['North Oconee (Stetson)']
Berkeley Prep: ['Berkeley Prep']
Windermere (Miami): ['Windermere (Miami)']
Wiregrass Ranch (USF): ['Wiregrass Ranch (USF)']
Marjory Stoneman Douglas: ['Marjory Stoneman Douglas']
Milton: ['Milton']
Bloomingdale (Alabama): ['Bloomingdale (Alabama)']
P27 Academy: ['P27 Academy']
Calvary Christian: ['Calvary Christian']
Parkview (Texas Tech): ['Parkview (Texas Tech)']
A3 Academy: ['A3 Academy']
Sickles: ['Sickles']
Verona: ['Verona']
Brookwood: ['Brookwood']
North Myrtle Beach (Clemson): ['North Myrtle Beach (Clemson)']
Miami Palmetto: ['Miami Palmetto']
St. John's Country Day: ["St. John's Country Day"]
Durant: ['Durant']
Lake Brantley: ['Lake Brantley']
Lakeland Christian (Virginia Tech): ['Lakeland Christian (Virginia Tech)']
TNXL Academy (Charleston Southern): ['TNXL Academy (Charleston Southern)']
Doral Academy Charter: ['Doral 

In [7]:
hs_mapping = {}
for canonical, variants in unique_schools.items():
    for variant in variants:
        hs_mapping[variant] = canonical

players['High School'] = players['High School'].map(hs_mapping).fillna(players['High School'])

before_dedup = len(schools)
schools['name'] = schools['name'].map(hs_mapping).fillna(schools['name'])
schools.drop_duplicates(subset=['name'], inplace=True)
after_dedup = len(schools)

print(f"Deduplicated {len(high_schools) - len(unique_schools.keys())} high school names.")
print(f"Deduplicated {before_dedup - after_dedup} school names.")

Deduplicated 10 high school names.
Deduplicated 10 school names.


In [8]:
def dedup_coaches(coaches_df):
    # 1. John Smith at ABC Univ: Head Coach
    # 2. John Smith at ABC Univ: Recruiting Coordinator
    # => 1. John Smith at ABC Univ: Head Coach | Recruiting Coordinator
    coaches_clean = coaches_df.groupby(['Name', 'School'], as_index=False).agg({
        'Title': lambda x: ' | '.join(sorted(set(x)))
    })
    return coaches_clean

print(coaches.head())
coaches_clean = dedup_coaches(coaches)
print(coaches_clean)

                  School              Name                    Title
0  University Of Florida  Kevin O'Sullivan               Head Coach
1  University Of Florida   Chuck Jeroloman     Associate Head Coach
2  University Of Florida      Taylor Black          Assistant Coach
3  University Of Florida        David Kopp           Pitching Coach
4  University Of Florida     Michael Byrne  Student Assistant Coach
                 Name                   School  \
0       Adam  Pallone       Rutgers University   
1         Adam Nelson   University of Maryland   
2       Alec Crawford  University Of Minnesota   
3         Alex Corbin   University of Maryland   
4      Amanda Branson    University Of Alabama   
..                ...                      ...   
150  Tyler Schwalbach   University Of Oklahoma   
151     Vince Rinaldi   University Of Oklahoma   
152          Will Fox     Texas A&M University   
153       Zach Cronin    University Of Florida   
154  Zach Weatherford    University Of Ind